In [1]:
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import plotly.express as px
import dash_bootstrap_components as dbc

import pandas as pd

import lightgbm as lgb
import pandas as pd
import numpy as np

In [2]:
def get_team_stats(team_name, stats_file_path):
    stats_df = pd.read_csv(stats_file_path)
    team_row = stats_df[stats_df['team'] == team_name]
    if team_row.empty:
        raise ValueError(f"Команда '{team_name}' не найдена в статистике.")
    return team_row.iloc[0, 1:].values  # Возвращаем статистику без имени команды


def predict_match_winner(team1, team2, model_path, stats_file_path):
    # Получаем статистику команд
    stats_team1 = get_team_stats(team1, stats_file_path)
    stats_team2 = get_team_stats(team2, stats_file_path)
    
    # Пропускаем матч, если хотя бы одна из команд не найдена
    if stats_team1 is None or stats_team2 is None:
        print("Одна или обе команды не найдены в базе данных.")
        return None, None

    # Загружаем модель LightGBM
    model = lgb.Booster(model_file=model_path)

    # Разница в статистике
    diff_stats = stats_team1 - stats_team2

    # Преобразуем разницу статистик в формат, подходящий для LightGBM
    diff_stats = np.array(diff_stats).reshape(1, -1)

    # Прогнозируем вероятности для двух классов
    prob = model.predict(diff_stats)  # Вероятности для двух классов

    prob_team1 = prob[0]  # Вероятность победы первой команды
    prob_team2 = 1 - prob_team1  # Вероятность победы второй команды (1 - prob_team1)

    return prob_team1, prob_team2  # Возвращаем вероятности для обеих команд

In [3]:
df_mm = pd.read_csv('maps_stats.csv')
df_mm

,team,map,times played,wins,losses,total rounds played,rounds won,win percent,pistol rounds,pistol rounds won,pistol round win percent,ct round win percent,t round win percent
0,Vitality,Ancient,0,0,0,0,0,0.0,0,0,0.0,0.0,0.0
1,Vitality,Anubis,13,6,7,314,160,46.2,26,9,34.6,44.9,57.0
2,Vitality,Dust 2,19,13,6,385,223,68.4,38,23,60.5,52.0,63.1
3,Vitality,Mirage,14,11,3,356,198,78.6,28,15,53.6,57.3,53.9
4,Vitality,Inferno,14,7,7,309,163,50.0,28,16,57.1,50.9,55.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,Zero Tenacity,Dust 2,16,6,10,371,172,37.5,32,18,56.2,48.3,44.6
311,Zero Tenacity,Mirage,22,7,15,504,241,31.8,44,22,50.0,46.3,49.6
312,Zero Tenacity,Inferno,16,9,7,353,193,56.2,32,20,62.5,52.1,57.9
313,Zero Tenacity,Nuke,19,10,9,422,200,52.6,38,20,52.6,54.5,40.4


In [4]:
df_ss = pd.read_csv('stts_teams.csv')
df_ss

,team,maps played,wins,losses,total kills,total deaths,rounds played,k/d ratio
0,Vitality,82,55,27,6420,5828,1842,1.10
1,Natus Vincere,92,63,29,6881,6268,1989,1.10
2,Spirit,79,52,27,5707,5212,1659,1.09
3,MOUZ,83,50,33,6266,5851,1817,1.07
4,The MongolZ,86,53,33,6551,6111,1902,1.07
5,G2,111,71,40,8098,7656,2399,1.06
6,MIBR,117,66,51,8146,7866,2452,1.04
7,PARIVISION,85,45,40,6284,6063,1839,1.04
8,9 Pandas,92,45,47,6726,6632,2024,1.01
9,Monte,132,65,67,10238,9824,2975,1.04


In [5]:
Teams_Name = list(df_ss['team'].unique())
Teams_Name

['Vitality',
 'Natus Vincere',
 'Spirit',
 'MOUZ',
 'The MongolZ',
 'G2',
 'MIBR',
 'PARIVISION',
 '9 Pandas',
 'Monte',
 'Eternal Fire',
 'B8',
 'FaZe',
 'Nemiga',
 'Aurora',
 'Astralis',
 'Passion UA',
 'fnatic',
 '3DMAX',
 'Liquid',
 'Imperial',
 'HEROIC',
 'Sangal',
 'Complexity',
 'AMKAL',
 '9z',
 'paiN',
 'BLEED',
 'SAW',
 'BetBoom',
 'Virtus.pro',
 'MOUZ NXT',
 'ENCE',
 'Falcons',
 'BIG',
 'FURIA',
 'OG',
 'Sashi',
 'SINNERS',
 'Fluxo',
 'Rebels',
 'Ninjas in Pyjamas',
 'RED Canids',
 'GamerLegion',
 'Zero Tenacity']

In [6]:
dropdown_teams_options = [{'label': team, 'value': team} for team in Teams_Name]
dropdown_teams_options

[{'label': 'Vitality', 'value': 'Vitality'},
 {'label': 'Natus Vincere', 'value': 'Natus Vincere'},
 {'label': 'Spirit', 'value': 'Spirit'},
 {'label': 'MOUZ', 'value': 'MOUZ'},
 {'label': 'The MongolZ', 'value': 'The MongolZ'},
 {'label': 'G2', 'value': 'G2'},
 {'label': 'MIBR', 'value': 'MIBR'},
 {'label': 'PARIVISION', 'value': 'PARIVISION'},
 {'label': '9 Pandas', 'value': '9 Pandas'},
 {'label': 'Monte', 'value': 'Monte'},
 {'label': 'Eternal Fire', 'value': 'Eternal Fire'},
 {'label': 'B8', 'value': 'B8'},
 {'label': 'FaZe', 'value': 'FaZe'},
 {'label': 'Nemiga', 'value': 'Nemiga'},
 {'label': 'Aurora', 'value': 'Aurora'},
 {'label': 'Astralis', 'value': 'Astralis'},
 {'label': 'Passion UA', 'value': 'Passion UA'},
 {'label': 'fnatic', 'value': 'fnatic'},
 {'label': '3DMAX', 'value': '3DMAX'},
 {'label': 'Liquid', 'value': 'Liquid'},
 {'label': 'Imperial', 'value': 'Imperial'},
 {'label': 'HEROIC', 'value': 'HEROIC'},
 {'label': 'Sangal', 'value': 'Sangal'},
 {'label': 'Complexit

In [7]:
Maps_Name = list(df_mm['map'].unique())
Maps_Name

['Ancient', 'Anubis', 'Dust 2', 'Mirage', 'Inferno', 'Nuke', 'Vertigo']

In [8]:
dropdown_maps_options = []

In [9]:
dropdown_maps_options.append({'label': 'Общая статистика', 'value': 0})

In [10]:
dropdown_maps_options.extend([{'label': maps, 'value': maps} for maps in Maps_Name])
dropdown_maps_options

[{'label': 'Общая статистика', 'value': 0},
 {'label': 'Ancient', 'value': 'Ancient'},
 {'label': 'Anubis', 'value': 'Anubis'},
 {'label': 'Dust 2', 'value': 'Dust 2'},
 {'label': 'Mirage', 'value': 'Mirage'},
 {'label': 'Inferno', 'value': 'Inferno'},
 {'label': 'Nuke', 'value': 'Nuke'},
 {'label': 'Vertigo', 'value': 'Vertigo'}]

In [36]:
# Список каналов Twitch для выбора
channels = [
    {"label": "Sl4m", "value": "bysl4m"},
    {"label": "PGL", "value": "stewie2k"},
    {"label": "BB_CS", "value": "betboom_cs_B"},
]

In [38]:
app = dash.Dash(__name__, title='Веб сервис', external_stylesheets=[dbc.themes.BOOTSTRAP])

@app.server.after_request
def add_csp_headers(response):
    response.headers['Content-Security-Policy'] = (
        "frame-ancestors 'self' http://localhost:* https://localhost:*;"
    )
    return response

app.layout = html.Div([
    html.H1('Анализ исходов киберспортивных матчей', style={'display': 'flex', 'justifyContent': 'center', 'marginTop': '30px'}),

    html.Div(
    children=[
       
        html.Div(
            children=[
                html.H3(
                    'Просмотр командной статистики',
                    style={'textAlign': 'center', 'marginBottom': '20px'}
                ),
                html.Div(
                    children=[
                        dcc.Dropdown(
                            id='drop-down-teams',
                            style={'marginTop': '4px', 'width': '100%', 'height': '40px', 'marginBottom': '10px'},
                            placeholder="Выберите команду",
                            options=dropdown_teams_options
                        ),
                        dcc.Dropdown(
                            id='drop-down-maps',
                            style={'marginTop': '4px', 'width': '100%', 'height': '40px', 'marginBottom': '10px'},
                            placeholder="Выберите карту",
                            options=dropdown_maps_options
                        ),
                        dbc.Button(
                            'Вывести статистику',
                            id='prnt-stats',
                            disabled=False,
                            outline=True,
                            color="primary",
                            className="me-1",
                            style={'height': '40px', 'width': '50%', 'marginTop': '10px'}
                        ),
                    ],
                    style={
                        'width': '100%',
                        'display': 'flex',
                        'flexDirection': 'column',
                        'justifyContent': 'center',
                        'alignItems': 'center',
                        'padding': '10px',
                    },
                ),
            ],
            style={
                'width': '50%',
                'display': 'flex',
                'flexDirection': 'column',
                'alignItems': 'center',
                'padding': '10px',
            },
        ),
        # Правый блок
        # Правый блок с заголовком
html.Div(
    children=[
        html.H3(
            'Предсказание итога матча',
            style={'textAlign': 'center', 'marginBottom': '20px'}
        ),
        html.Div(
            children=[
                dcc.Dropdown(
                    id='drop-down-team1',
                    style={'marginTop': '4px', 'width': '100%', 'height': '40px', 'marginBottom': '10px'},
                    placeholder="Выберите команду",
                    options=dropdown_teams_options
                ),
                dcc.Dropdown(
                    id='drop-down-team2',
                    style={'marginTop': '4px', 'width': '100%', 'height': '40px', 'marginBottom': '10px'},
                    placeholder="Выберите команду",
                    options=dropdown_teams_options
                ),
                dbc.Button(
                    'Вывести результат',
                    id='prnt-result',
                    disabled=False,
                    outline=True,
                    color="primary",
                    className="me-1",
                    style={'height': '40px', 'width': '50%', 'marginTop': '10px'}
                ),
            ],
            style={
                'width': '100%',
                'display': 'flex',
                'flexDirection': 'column',
                'justifyContent': 'center',
                'alignItems': 'center',
                'padding': '10px',
            },
        ),
    ],
    style={
        'width': '50%',
        'display': 'flex',
        'flexDirection': 'column',
        'alignItems': 'center',
        'padding': '10px',
    },
),

    ],
    style={
        'display': 'flex',
        'flexDirection': 'row',
        'justifyContent': 'space-between',
        'alignItems': 'flex-start',
        'width': '100%',
        'padding': '20px',
    },
),

    
    html.Div([
        html.Div(id="output-stat-team", style={'width': '50%', 'display': 'inline-block', 'verticalAlign': 'middle', 'marginRight': '10px', 'marginLeft': '25px', 'marginTop': '15px'}),
        
        html.Div(id='output-result', style={'width': '50%', 'display': 'inline-block', 'verticalAlign': 'middle', 'marginRight': '25px', 'marginLeft': '10px', 'marginTop': '15px'})
    ], style={'display': 'flex', 'flexWrap': 'nowrap', 'height': '80%'}),
    # [dcc.Graph(id='function-output2')]

    html.Div(
            children=[
                # Dropdown для выбора канала
                dcc.Dropdown(
                    id="channel-dropdown",
                    value=channels[1]['value'],
                    options=channels,
                    placeholder="Выберите Twitch-канал",
                    style={"width": "300px", "marginBottom": "20px"}
                ),
                html.Iframe(
                    id="twitch-iframe",
                    src="https://player.twitch.tv/?channel=koreshzy&parent=localhost&parent=127.0.0.1",
                    style={
                        "width": "1024px",
                        "height": "600px",
                        "border": "3px solid black",  # Добавляет границу толщиной 5px и черного цвета
                        "borderRadius": "10px"  # Скругленные углы границы (опционально)
                    }
                )
            ],
            style={
                "display": "flex",
                "justifyContent": "center",
                "alignItems": "center",
                "flexDirection": "column",
                "textAlign": "center",  # Центрирование текста внутри элементов
                "padding": "20px"
            }
        )
         
])


@app.callback(
    [Output("output-stat-team", "children")],
    Input('prnt-stats', 'n_clicks'),
    [State('drop-down-teams', "value"),
     State('drop-down-maps', "value")]
)

def call_status_system(btn1, dropdwn1,  dropdwn2):
    ctx = dash.callback_context

    if not ctx.triggered:
        return [""]

    button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if button_id == "prnt-stats":
        if dropdwn1 is None:
            return [html.Div("Команда не выбрана", style={'color': 'red', 'margin': '10px', 'fontSize': '20px', 'textAlign': 'center'})]

        if dropdwn2 is None:
            return [html.Div("Карта не выбрана", style={'color': 'red', 'margin': '10px', 'fontSize': '20px', 'textAlign': 'center'})]

        if dropdwn2 == 0:
            name = df_ss[df_ss["team"] == dropdwn1]['team'].tolist()[0]
            maps = df_ss[df_ss["team"] == dropdwn1]['maps played'].tolist()[0]
            win = df_ss[df_ss["team"] == dropdwn1]['wins'].tolist()[0]
            loss = df_ss[df_ss["team"] == dropdwn1]['losses'].tolist()[0]
            kills = df_ss[df_ss["team"] == dropdwn1]['total kills'].tolist()[0]
            deaths = df_ss[df_ss["team"] == dropdwn1]['total deaths'].tolist()[0]
            rounds = df_ss[df_ss["team"] == dropdwn1]['rounds played'].tolist()[0]
            kd = df_ss[df_ss["team"] == dropdwn1]['k/d ratio'].tolist()[0]
            
            # Заранее заготовленные строки
            pre_defined_rows = [
                f"Название команды: {name}",
                f"Сыграно карт: {maps}",
                f"Количество побед: {win}",
                f"Количество поражений: {loss}",
                f"Количество убийств: {kills}",
                f"Количество смертей: {deaths}",
                f"Сыграно раундов: {rounds}",
                f"Рейтинг KD: {kd}"
            ]
    
            if dropdwn1 == 'Vitality':
                image_url = "https://img-cdn.hltv.org/teamlogo/yeXBldn9w8LZCgdElAenPs.png?ixlib=java-2.1.0&w=200&s=1803f5a02a92e431b9b7d8239fa15d47"
            elif dropdwn1 == 'Natus Vincere':
                image_url = "https://img-cdn.hltv.org/teamlogo/9iMirAi7ArBLNU8p3kqUTZ.svg?ixlib=java-2.1.0&s=4dd8635be16122656093ae9884675d0c"
            elif dropdwn1 == 'Spirit':
                image_url = "https://img-cdn.hltv.org/teamlogo/ywdn4tmAvXfllLeV2SkkvF.png?ixlib=java-2.1.0&w=200&s=4878ec4edb411cf0c348226b0798d14f"
            elif dropdwn1 == 'MOUZ':
                image_url = "https://img-cdn.hltv.org/teamlogo/IejtXpquZnE8KqYPB1LNKw.svg?ixlib=java-2.1.0&s=7fd33b8def053fbfd8fdbb58e3bdcd3c"
            elif dropdwn1 == 'The MongolZ':
                image_url = "https://img-cdn.hltv.org/teamlogo/bRk2sh_tSTO6fq1GLhgcal.png?ixlib=java-2.1.0&w=200&s=d82e930fcea873b51ceab34c1a338b02"
            elif dropdwn1 == 'G2':
                image_url = "https://img-cdn.hltv.org/teamlogo/zFLwAELOD15BjJSDMMNBWQ.png?ixlib=java-2.1.0&w=200&s=457c1663356d6dd20e39a1188b267802"
            elif dropdwn1 == 'MIBR':
                image_url = "https://img-cdn.hltv.org/teamlogo/m_JQ624LNFHWiUY-25uuaE.png?ixlib=java-2.1.0&w=200&s=a1967c33cfc061974950f14fea540d13"
            elif dropdwn1 == 'PARIVISION':
                image_url = "https://img-cdn.hltv.org/teamlogo/I4OJxeJAFmZ222Ki0lYTd9.png?ixlib=java-2.1.0&w=200&s=b08dec3a22307c89af8720a5679abaf9"
            elif dropdwn1 == '9 Pandas':
                image_url = "https://img-cdn.hltv.org/teamlogo/u4ZlTBpqD-bXOZRqxYY385.png?ixlib=java-2.1.0&w=200&s=dccffbeb52fcbadf29dd2d7ece34f557"
            elif dropdwn1 == 'Monte':
                image_url = "https://img-cdn.hltv.org/teamlogo/2tc9n4fHkiRIX2FiJSkhgt.png?ixlib=java-2.1.0&w=200&s=52c60e51e8c1797c3101eb339cdf0d24"
            elif dropdwn1 == 'Eternal Fire':
                image_url = "https://img-cdn.hltv.org/teamlogo/Tafdq71X3B_-73b73bAixr.png?ixlib=java-2.1.0&w=200&s=9971096d8840611576c29d89b5985e7b"
            elif dropdwn1 == 'B8':
                image_url = "https://img-cdn.hltv.org/teamlogo/rUIL5D299QDtRjXR2rxqV_.png?ixlib=java-2.1.0&w=200&s=769cb3413dd6e09e1a042622fc5a2568"
            elif dropdwn1 == 'FaZe':
                image_url = "https://img-cdn.hltv.org/teamlogo/m-wCSia4m35BeWhGzB9yv1.png?ixlib=java-2.1.0&w=200&s=879e2c21b0bc0a90ebe9af3d38d8dee6"
            elif dropdwn1 == 'Nemiga':
                image_url = "https://img-cdn.hltv.org/teamlogo/6ZPCm5r3XyGByXkFGiRnLh.png?ixlib=java-2.1.0&w=200&s=1a914d0f4164219aee64839b9002d22c"
            elif dropdwn1 == 'Aurora':
                image_url = "https://img-cdn.hltv.org/teamlogo/Tn34gRpdxo9yhixcQSnpMP.png?ixlib=java-2.1.0&w=200&s=7090ae651d67c684fc46cba650e83a3d"
            elif dropdwn1 == 'Astralis':
                image_url = "https://img-cdn.hltv.org/teamlogo/9bgXHp-oh1oaXr7F0mTGmd.svg?ixlib=java-2.1.0&s=f567161ab183001be33948b98c4b2067"
            elif dropdwn1 == 'Passion UA':
                image_url = "https://img-cdn.hltv.org/teamlogo/3Ln7TXA0zyo4AA-RXglS8_.png?ixlib=java-2.1.0&w=200&s=d67a301387d60230293f1d40a592c986"
            elif dropdwn1 == 'fnatic':
                image_url = "https://img-cdn.hltv.org/teamlogo/dLtWEdSV58lIX1amAFggy0.svg?ixlib=java-2.1.0&s=f24d0a7b3ef24ed57184a51d35202b4e"
            elif dropdwn1 == '3DMAX':
                image_url = "https://img-cdn.hltv.org/teamlogo/uZYa5_FFR-SxtqMQxNGzco.svg?ixlib=java-2.1.0&s=6e7c8a89950b732529e481338b5892b8"
            elif dropdwn1 == 'Liquid':
                image_url = "https://img-cdn.hltv.org/teamlogo/JMeLLbWKCIEJrmfPaqOz4O.svg?ixlib=java-2.1.0&s=c02caf90234d3a3ebac074c84ba1ea62"
            elif dropdwn1 == 'Imperial':
                image_url = "https://img-cdn.hltv.org/teamlogo/WF5P55R5fxcmhoew6vHZQr.png?ixlib=java-2.1.0&w=200&s=993151de8af78c610d7b6da11ac954b9"
            elif dropdwn1 == 'HEROIC':
                image_url = "https://img-cdn.hltv.org/teamlogo/6LVaK1MVX06kO1ChAq22v6.svg?ixlib=java-2.1.0&s=1ad0be7366aab3ef6825996762c3e615"
            elif dropdwn1 == 'Sangal':
                image_url = "https://img-cdn.hltv.org/teamlogo/zPv_FeMF8CANC10Jz32P9l.png?ixlib=java-2.1.0&w=200&s=2f9482695b2da7dcb380010c1a23b3b5"
            elif dropdwn1 == 'Complexity':
                image_url = "https://img-cdn.hltv.org/teamlogo/0-i_bEjrf3v4eYqaG0Bix7.svg?ixlib=java-2.1.0&s=4eecbec277f018772a9b92c22da1a459"
            elif dropdwn1 == 'AMKAL':
                image_url = "https://img-cdn.hltv.org/teamlogo/KNl9ZeR2afmpj3-J0fSilz.png?ixlib=java-2.1.0&w=200&s=0a43afad15e8ed5e080afbd3e9ce2524"
            elif dropdwn1 == '9z':
                image_url = "https://img-cdn.hltv.org/teamlogo/COZDFWOIm41AT0srqOHFhM.png?ixlib=java-2.1.0&w=200&s=c782dafac785c6eef7e0aa51f2c7f5aa"
            elif dropdwn1 == 'paiN':
                image_url = "https://img-cdn.hltv.org/teamlogo/x7znFH8_QjLXOQRyR2y7Xp.png?ixlib=java-2.1.0&w=200&s=f01eb19584b41def8740941f97084fc7"
            elif dropdwn1 == 'BLEED':
                image_url = "https://img-cdn.hltv.org/teamlogo/OEEjDTpY3cEk3cFTQ4TT4b.png?ixlib=java-2.1.0&w=200&s=337b779b843f43160c1d63034847c30a"
            elif dropdwn1 == 'SAW':
                image_url = "https://img-cdn.hltv.org/teamlogo/9vOlYp2U_z0vXPb9aLK-4r.png?invert=true&ixlib=java-2.1.0&sat=-100&w=200&s=6c90941e057f990a660142d187cb730f"
            elif dropdwn1 == 'BetBoom':
                image_url = "https://img-cdn.hltv.org/teamlogo/G4ZrdB0-q41USPd_z27IQA.png?ixlib=java-2.1.0&w=200&s=ffff79219e7490953d9d79771fcf8193"
            elif dropdwn1 == 'Virtus.pro':
                image_url = "https://img-cdn.hltv.org/teamlogo/yZ6Bpuui1rW3jocXQ68XgZ.svg?ixlib=java-2.1.0&s=f39be1d3e7baf30a4e7f0b1216720875"
            elif dropdwn1 == 'MOUZ NXT':
                image_url = "https://img-cdn.hltv.org/teamlogo/RfR1zmFJ0eP08VmFb6UOu3.png?ixlib=java-2.1.0&w=200&s=33bffc78a47242023aa33c7950ce7d4a"
            elif dropdwn1 == 'ENCE':
                image_url = "https://img-cdn.hltv.org/teamlogo/-X8NoyWC_1gYqUHvZqcpkc.svg?ixlib=java-2.1.0&s=85bb9daa6f846fa097c5942f2565fdb8"
            elif dropdwn1 == 'Falcons':
                image_url = "https://img-cdn.hltv.org/teamlogo/4eJSkDQINNM6Tbs4WvLzkN.png?ixlib=java-2.1.0&w=200&s=d4dd034c2c7786f7c75e51a3a3aa7b5c"
            elif dropdwn1 == 'BIG':
                image_url = "https://img-cdn.hltv.org/teamlogo/yQB6cm3KZ_BcyrgppBQMjc.svg?ixlib=java-2.1.0&s=06825290bfb61c9f8467f5c323f51974"
            elif dropdwn1 == 'FURIA':
                image_url = "https://img-cdn.hltv.org/teamlogo/mvNQc4csFGtxXk5guAh8m1.svg?ixlib=java-2.1.0&s=11e5056829ad5d6c06c5961bbe76d20c"
            elif dropdwn1 == 'OG':
                image_url = "https://img-cdn.hltv.org/teamlogo/VZHA__RIJY6gisB-l6HfiO.png?ixlib=java-2.1.0&w=200&s=eba028bb9a778e011fd92be6831cad5a"
            elif dropdwn1 == 'Sashi':
                image_url = "https://img-cdn.hltv.org/teamlogo/xa5M31PECYUvHs6wlM9nzM.png?ixlib=java-2.1.0&w=200&s=e6b46fef96ff79a105038e3c063a1fa8"
            elif dropdwn1 == 'SINNERS':
                image_url = "https://img-cdn.hltv.org/teamlogo/9l_WdQSU9JsNHzpK-pwOG2.svg?ixlib=java-2.1.0&s=af432c3ef61c0c843331cc0dc2fed1ed"
            elif dropdwn1 == 'Fluxo':
                image_url = "https://img-cdn.hltv.org/teamlogo/zGveLNj821BCJGMu93qnj-.png?invert=true&ixlib=java-2.1.0&sat=-100&w=200&s=3f935421499d4a1303b77c6ec83f5bf9"
            elif dropdwn1 == 'Rebels':
                image_url = "https://img-cdn.hltv.org/teamlogo/vfupmnriFumqC8u48wFIdm.png?ixlib=java-2.1.0&w=200&s=71b0e8f8fb6bf62750e107d79ec5f92c"
            elif dropdwn1 == 'Ninjas in Pyjamas':
                image_url = "https://img-cdn.hltv.org/teamlogo/mbY2YeFKUAbL9NyO6ab7-c.png?ixlib=java-2.1.0&w=200&s=1c4343c7ff32e9e75f9321b91d443d07"
            elif dropdwn1 == 'RED Canids':
                image_url = "https://img-cdn.hltv.org/teamlogo/Lyr7VJ-litGOwnRBu_8K5q.png?ixlib=java-2.1.0&w=200&s=017c11d0ceabde9b5ed892c789a71c3d"
            elif dropdwn1 == 'GamerLegion':
                image_url = "https://img-cdn.hltv.org/teamlogo/jS__cj2F09Bl8qBU_CvkQR.png?ixlib=java-2.1.0&w=200&s=9b9252b6e3737f4a32c1de457bc308ce"
            elif dropdwn1 == 'Zero Tenacity':
                image_url = "https://img-cdn.hltv.org/teamlogo/xH4vGSdADi6Dz62BA9_fQ3.png?ixlib=java-2.1.0&w=200&s=92a68b14e42f8eebebe5799710027c1f"
            else:
                image_url = "https://img-cdn.hltv.org/teamlogo/default.png"  # URL для случая, если команда не найдена   
    
            
            # Создаём HTML Div элементы для каждой строки
            div_elements = [
                html.Div(row, style={'margin': '5px', 'fontSize': '20px', 'textAlign': 'center'})
                for row in pre_defined_rows
            ]
    
            # Добавляем изображение
            # image_element = html.Img(src=image_url, style={'width': '150px', 'height': '150px', 'display': 'block', 'margin': '10px auto', 'background-color': 'lightblue'})

            image_element = html.Img(
                src=image_url,
                style={
                    'width': '150px',
                    'height': '150px',
                    'display': 'block',
                    'margin': '10px auto',
                    'border': '2px solid black',  # Границы для выделения изображения
                    'borderRadius': '10px',  # Скруглённые углы для более красивого эффекта
                    'padding': '5px',  # Внутренний отступ между изображением и границей
                    'boxShadow': '0px 4px 10px rgba(0, 0, 0, 0.5)',  # Добавлена лёгкая тень
                    'backgroundColor': '#b3b3b3'
                }
            )

            
            # Возвращаем центрально выровненный Div с текстом и изображением
            return [html.Div([image_element] + div_elements, style={'textAlign': 'center'})]
        else:

            name = df_mm[(df_mm["team"] == dropdwn1) & (df_mm["map"] == dropdwn2)]['team'].tolist()[0]
            maps = df_mm[(df_mm["team"] == dropdwn1) & (df_mm["map"] == dropdwn2)]['map'].tolist()[0]
            times_played = df_mm[(df_mm["team"] == dropdwn1) & (df_mm["map"] == dropdwn2)]['times played'].tolist()[0]
            wins = df_mm[(df_mm["team"] == dropdwn1) & (df_mm["map"] == dropdwn2)]['wins'].tolist()[0]
            losses = df_mm[(df_mm["team"] == dropdwn1) & (df_mm["map"] == dropdwn2)]['losses'].tolist()[0]
            total_round_played = df_mm[(df_mm["team"] == dropdwn1) & (df_mm["map"] == dropdwn2)]['total rounds played'].tolist()[0]
            rounds_won = df_mm[(df_mm["team"] == dropdwn1) & (df_mm["map"] == dropdwn2)]['rounds won'].tolist()[0]
            win_percent = df_mm[(df_mm["team"] == dropdwn1) & (df_mm["map"] == dropdwn2)]['win percent'].tolist()[0]
            pistol_rounds = df_mm[(df_mm["team"] == dropdwn1) & (df_mm["map"] == dropdwn2)]['pistol rounds'].tolist()[0]
            pistol_rounds_won = df_mm[(df_mm["team"] == dropdwn1) & (df_mm["map"] == dropdwn2)]['pistol rounds won'].tolist()[0]
            pistol_rounds_win_percent = df_mm[(df_mm["team"] == dropdwn1) & (df_mm["map"] == dropdwn2)]['pistol round win percent'].tolist()[0]
            ct_round_win_percent = df_mm[(df_mm["team"] == dropdwn1) & (df_mm["map"] == dropdwn2)]['ct round win percent'].tolist()[0]
            t_round_win_percent = df_mm[(df_mm["team"] == dropdwn1) & (df_mm["map"] == dropdwn2)]['t round win percent'].tolist()[0]
            
            # Заранее заготовленные строки
            pre_defined_rows = [
                f"Название команды: {name}",
                f"Название карты: {maps}",
                f"Количество игр: {times_played}",
                f"Количество побед: {wins}",
                f"Количество поражений: {losses}",
                f"Сыграно раундов: {total_round_played}",
                f"Выиграно раундов: {rounds_won}",
                f"Процент побед: {win_percent}",
                f"Количество пистолетных раундов: {pistol_rounds}",
                f"Побед на пистолетных раундах: {pistol_rounds_won}",
                f"Процент побед на пистолетных раундах: {pistol_rounds_win_percent}",
                f"Процент побед за КТ: {ct_round_win_percent}",
                f"Процент побед за Т: {t_round_win_percent}"
            ]            
            
            if dropdwn1 == 'Vitality':
                image_url = "https://img-cdn.hltv.org/teamlogo/yeXBldn9w8LZCgdElAenPs.png?ixlib=java-2.1.0&w=200&s=1803f5a02a92e431b9b7d8239fa15d47"
            elif dropdwn1 == 'Natus Vincere':
                image_url = "https://img-cdn.hltv.org/teamlogo/9iMirAi7ArBLNU8p3kqUTZ.svg?ixlib=java-2.1.0&s=4dd8635be16122656093ae9884675d0c"
            elif dropdwn1 == 'Spirit':
                image_url = "https://img-cdn.hltv.org/teamlogo/ywdn4tmAvXfllLeV2SkkvF.png?ixlib=java-2.1.0&w=200&s=4878ec4edb411cf0c348226b0798d14f"
            elif dropdwn1 == 'MOUZ':
                image_url = "https://img-cdn.hltv.org/teamlogo/IejtXpquZnE8KqYPB1LNKw.svg?ixlib=java-2.1.0&s=7fd33b8def053fbfd8fdbb58e3bdcd3c"
            elif dropdwn1 == 'The MongolZ':
                image_url = "https://img-cdn.hltv.org/teamlogo/bRk2sh_tSTO6fq1GLhgcal.png?ixlib=java-2.1.0&w=200&s=d82e930fcea873b51ceab34c1a338b02"
            elif dropdwn1 == 'G2':
                image_url = "https://img-cdn.hltv.org/teamlogo/zFLwAELOD15BjJSDMMNBWQ.png?ixlib=java-2.1.0&w=200&s=457c1663356d6dd20e39a1188b267802"
            elif dropdwn1 == 'MIBR':
                image_url = "https://img-cdn.hltv.org/teamlogo/m_JQ624LNFHWiUY-25uuaE.png?ixlib=java-2.1.0&w=200&s=a1967c33cfc061974950f14fea540d13"
            elif dropdwn1 == 'PARIVISION':
                image_url = "https://img-cdn.hltv.org/teamlogo/I4OJxeJAFmZ222Ki0lYTd9.png?ixlib=java-2.1.0&w=200&s=b08dec3a22307c89af8720a5679abaf9"
            elif dropdwn1 == '9 Pandas':
                image_url = "https://img-cdn.hltv.org/teamlogo/u4ZlTBpqD-bXOZRqxYY385.png?ixlib=java-2.1.0&w=200&s=dccffbeb52fcbadf29dd2d7ece34f557"
            elif dropdwn1 == 'Monte':
                image_url = "https://img-cdn.hltv.org/teamlogo/2tc9n4fHkiRIX2FiJSkhgt.png?ixlib=java-2.1.0&w=200&s=52c60e51e8c1797c3101eb339cdf0d24"
            elif dropdwn1 == 'Eternal Fire':
                image_url = "https://img-cdn.hltv.org/teamlogo/Tafdq71X3B_-73b73bAixr.png?ixlib=java-2.1.0&w=200&s=9971096d8840611576c29d89b5985e7b"
            elif dropdwn1 == 'B8':
                image_url = "https://img-cdn.hltv.org/teamlogo/rUIL5D299QDtRjXR2rxqV_.png?ixlib=java-2.1.0&w=200&s=769cb3413dd6e09e1a042622fc5a2568"
            elif dropdwn1 == 'FaZe':
                image_url = "https://img-cdn.hltv.org/teamlogo/m-wCSia4m35BeWhGzB9yv1.png?ixlib=java-2.1.0&w=200&s=879e2c21b0bc0a90ebe9af3d38d8dee6"
            elif dropdwn1 == 'Nemiga':
                image_url = "https://img-cdn.hltv.org/teamlogo/6ZPCm5r3XyGByXkFGiRnLh.png?ixlib=java-2.1.0&w=200&s=1a914d0f4164219aee64839b9002d22c"
            elif dropdwn1 == 'Aurora':
                image_url = "https://img-cdn.hltv.org/teamlogo/Tn34gRpdxo9yhixcQSnpMP.png?ixlib=java-2.1.0&w=200&s=7090ae651d67c684fc46cba650e83a3d"
            elif dropdwn1 == 'Astralis':
                image_url = "https://img-cdn.hltv.org/teamlogo/9bgXHp-oh1oaXr7F0mTGmd.svg?ixlib=java-2.1.0&s=f567161ab183001be33948b98c4b2067"
            elif dropdwn1 == 'Passion UA':
                image_url = "https://img-cdn.hltv.org/teamlogo/3Ln7TXA0zyo4AA-RXglS8_.png?ixlib=java-2.1.0&w=200&s=d67a301387d60230293f1d40a592c986"
            elif dropdwn1 == 'fnatic':
                image_url = "https://img-cdn.hltv.org/teamlogo/dLtWEdSV58lIX1amAFggy0.svg?ixlib=java-2.1.0&s=f24d0a7b3ef24ed57184a51d35202b4e"
            elif dropdwn1 == '3DMAX':
                image_url = "https://img-cdn.hltv.org/teamlogo/uZYa5_FFR-SxtqMQxNGzco.svg?ixlib=java-2.1.0&s=6e7c8a89950b732529e481338b5892b8"
            elif dropdwn1 == 'Liquid':
                image_url = "https://img-cdn.hltv.org/teamlogo/JMeLLbWKCIEJrmfPaqOz4O.svg?ixlib=java-2.1.0&s=c02caf90234d3a3ebac074c84ba1ea62"
            elif dropdwn1 == 'Imperial':
                image_url = "https://img-cdn.hltv.org/teamlogo/WF5P55R5fxcmhoew6vHZQr.png?ixlib=java-2.1.0&w=200&s=993151de8af78c610d7b6da11ac954b9"
            elif dropdwn1 == 'HEROIC':
                image_url = "https://img-cdn.hltv.org/teamlogo/6LVaK1MVX06kO1ChAq22v6.svg?ixlib=java-2.1.0&s=1ad0be7366aab3ef6825996762c3e615"
            elif dropdwn1 == 'Sangal':
                image_url = "https://img-cdn.hltv.org/teamlogo/zPv_FeMF8CANC10Jz32P9l.png?ixlib=java-2.1.0&w=200&s=2f9482695b2da7dcb380010c1a23b3b5"
            elif dropdwn1 == 'Complexity':
                image_url = "https://img-cdn.hltv.org/teamlogo/0-i_bEjrf3v4eYqaG0Bix7.svg?ixlib=java-2.1.0&s=4eecbec277f018772a9b92c22da1a459"
            elif dropdwn1 == 'AMKAL':
                image_url = "https://img-cdn.hltv.org/teamlogo/KNl9ZeR2afmpj3-J0fSilz.png?ixlib=java-2.1.0&w=200&s=0a43afad15e8ed5e080afbd3e9ce2524"
            elif dropdwn1 == '9z':
                image_url = "https://img-cdn.hltv.org/teamlogo/COZDFWOIm41AT0srqOHFhM.png?ixlib=java-2.1.0&w=200&s=c782dafac785c6eef7e0aa51f2c7f5aa"
            elif dropdwn1 == 'paiN':
                image_url = "https://img-cdn.hltv.org/teamlogo/x7znFH8_QjLXOQRyR2y7Xp.png?ixlib=java-2.1.0&w=200&s=f01eb19584b41def8740941f97084fc7"
            elif dropdwn1 == 'BLEED':
                image_url = "https://img-cdn.hltv.org/teamlogo/OEEjDTpY3cEk3cFTQ4TT4b.png?ixlib=java-2.1.0&w=200&s=337b779b843f43160c1d63034847c30a"
            elif dropdwn1 == 'SAW':
                image_url = "https://img-cdn.hltv.org/teamlogo/9vOlYp2U_z0vXPb9aLK-4r.png?invert=true&ixlib=java-2.1.0&sat=-100&w=200&s=6c90941e057f990a660142d187cb730f"
            elif dropdwn1 == 'BetBoom':
                image_url = "https://img-cdn.hltv.org/teamlogo/G4ZrdB0-q41USPd_z27IQA.png?ixlib=java-2.1.0&w=200&s=ffff79219e7490953d9d79771fcf8193"
            elif dropdwn1 == 'Virtus.pro':
                image_url = "https://img-cdn.hltv.org/teamlogo/yZ6Bpuui1rW3jocXQ68XgZ.svg?ixlib=java-2.1.0&s=f39be1d3e7baf30a4e7f0b1216720875"
            elif dropdwn1 == 'MOUZ NXT':
                image_url = "https://img-cdn.hltv.org/teamlogo/RfR1zmFJ0eP08VmFb6UOu3.png?ixlib=java-2.1.0&w=200&s=33bffc78a47242023aa33c7950ce7d4a"
            elif dropdwn1 == 'ENCE':
                image_url = "https://img-cdn.hltv.org/teamlogo/-X8NoyWC_1gYqUHvZqcpkc.svg?ixlib=java-2.1.0&s=85bb9daa6f846fa097c5942f2565fdb8"
            elif dropdwn1 == 'Falcons':
                image_url = "https://img-cdn.hltv.org/teamlogo/4eJSkDQINNM6Tbs4WvLzkN.png?ixlib=java-2.1.0&w=200&s=d4dd034c2c7786f7c75e51a3a3aa7b5c"
            elif dropdwn1 == 'BIG':
                image_url = "https://img-cdn.hltv.org/teamlogo/yQB6cm3KZ_BcyrgppBQMjc.svg?ixlib=java-2.1.0&s=06825290bfb61c9f8467f5c323f51974"
            elif dropdwn1 == 'FURIA':
                image_url = "https://img-cdn.hltv.org/teamlogo/mvNQc4csFGtxXk5guAh8m1.svg?ixlib=java-2.1.0&s=11e5056829ad5d6c06c5961bbe76d20c"
            elif dropdwn1 == 'OG':
                image_url = "https://img-cdn.hltv.org/teamlogo/VZHA__RIJY6gisB-l6HfiO.png?ixlib=java-2.1.0&w=200&s=eba028bb9a778e011fd92be6831cad5a"
            elif dropdwn1 == 'Sashi':
                image_url = "https://img-cdn.hltv.org/teamlogo/xa5M31PECYUvHs6wlM9nzM.png?ixlib=java-2.1.0&w=200&s=e6b46fef96ff79a105038e3c063a1fa8"
            elif dropdwn1 == 'SINNERS':
                image_url = "https://img-cdn.hltv.org/teamlogo/9l_WdQSU9JsNHzpK-pwOG2.svg?ixlib=java-2.1.0&s=af432c3ef61c0c843331cc0dc2fed1ed"
            elif dropdwn1 == 'Fluxo':
                image_url = "https://img-cdn.hltv.org/teamlogo/zGveLNj821BCJGMu93qnj-.png?invert=true&ixlib=java-2.1.0&sat=-100&w=200&s=3f935421499d4a1303b77c6ec83f5bf9"
            elif dropdwn1 == 'Rebels':
                image_url = "https://img-cdn.hltv.org/teamlogo/vfupmnriFumqC8u48wFIdm.png?ixlib=java-2.1.0&w=200&s=71b0e8f8fb6bf62750e107d79ec5f92c"
            elif dropdwn1 == 'Ninjas in Pyjamas':
                image_url = "https://img-cdn.hltv.org/teamlogo/mbY2YeFKUAbL9NyO6ab7-c.png?ixlib=java-2.1.0&w=200&s=1c4343c7ff32e9e75f9321b91d443d07"
            elif dropdwn1 == 'RED Canids':
                image_url = "https://img-cdn.hltv.org/teamlogo/Lyr7VJ-litGOwnRBu_8K5q.png?ixlib=java-2.1.0&w=200&s=017c11d0ceabde9b5ed892c789a71c3d"
            elif dropdwn1 == 'GamerLegion':
                image_url = "https://img-cdn.hltv.org/teamlogo/jS__cj2F09Bl8qBU_CvkQR.png?ixlib=java-2.1.0&w=200&s=9b9252b6e3737f4a32c1de457bc308ce"
            elif dropdwn1 == 'Zero Tenacity':
                image_url = "https://img-cdn.hltv.org/teamlogo/xH4vGSdADi6Dz62BA9_fQ3.png?ixlib=java-2.1.0&w=200&s=92a68b14e42f8eebebe5799710027c1f"
            else:
                image_url = "https://img-cdn.hltv.org/teamlogo/default.png"  # URL для случая, если команда не найдена


            # Разделяем строки на два столбца
            column_1 = pre_defined_rows[:len(pre_defined_rows)//2]
            column_2 = pre_defined_rows[len(pre_defined_rows)//2:]
            
            # Создаём Div элементы для каждой строки
            column_1_divs = [
                html.Div(row, style={'margin': '5px', 'fontSize': '20px', 'textAlign': 'left'})
                for row in column_1
            ]
            
            column_2_divs = [
                html.Div(row, style={'margin': '5px', 'fontSize': '20px', 'textAlign': 'left'})
                for row in column_2
            ]
            
            # Создаём основной Div с двумя столбцами
            two_columns_div = html.Div(
                children=[
                    html.Div(column_1_divs, style={'width': '50%', 'padding': '10px'}),
                    html.Div(column_2_divs, style={'width': '50%', 'padding': '10px'})
                ],
                style={
                    'display': 'flex',
                    'justifyContent': 'space-between',
                    'alignItems': 'flex-start',
                    'width': '100%'
                }
            )
            
            
            # image_element = html.Img(
            #     src=image_url,
            #     style={
            #         'width': '150px',
            #         'height': '150px',
            #         'display': 'block',
            #         'margin': '10px auto',
            #         'backgroundColor': 'lightblue'
            #     }
            # )

            image_element = html.Img(
                src=image_url,
                style={
                    'width': '150px',
                    'height': '150px',
                    'display': 'block',
                    'margin': '10px auto',
                    'border': '2px solid black',  # Границы для выделения изображения
                    'borderRadius': '10px',  # Скруглённые углы для более красивого эффекта
                    'padding': '5px',  # Внутренний отступ между изображением и границей
                    'boxShadow': '0px 4px 10px rgba(0, 0, 0, 0.5)',  # Добавлена лёгкая тень
                    'backgroundColor': '#b3b3b3'
                }
            )
            
            # Возвращаем общий контейнер с изображением и двумя столбцами
            return [
                html.Div(
                    children=[image_element, two_columns_div],
                    style={'textAlign': 'center'}
                )
            ]
            
            
@app.callback(
    [Output("output-result", "children")],
    Input('prnt-result', 'n_clicks'),
    [State('drop-down-team1', "value"),
     State('drop-down-team2', "value")]
)

def call_status_system(btn1, dropdwn1, dropdwn2):
    ctx = dash.callback_context

    if not ctx.triggered:
        return [""]

    button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if button_id == "prnt-result":
        if dropdwn1 is None:
            return [html.Div("Команда не выбрана", style={'color': 'red', 'margin': '10px', 'fontSize': '20px', 'textAlign': 'center'})]
   
    model_path = 'prediction_model_last_lgmb.txt'  # Путь к сохраненной модели LightGBM
    stats_file_path = 'Статистика команд.csv'  # Путь к CSV файлу с данными статистики команд
    
    # Пример команд
    team1 = dropdwn1
    team2 = dropdwn2

    if team1 == None or team2 == None:
        return [html.Div("Команда не выбрана", style={'color': 'red', 'margin': '10px', 'fontSize': '20px', 'textAlign': 'center'})]

    if team1 == team2:
        return [html.Div("Выбраны одинаковые команнды", style={'color': 'red', 'margin': '10px', 'fontSize': '20px', 'textAlign': 'center'})]
    
    # Получаем вероятности победы
    prob_team1, prob_team2 = predict_match_winner(team1, team2, model_path, stats_file_path)

    prob_text = [
            html.Div(f"Вероятность победы {team1}: {prob_team1:.5f}", style={'fontSize': '20px', 'margin': '10px'}),
            html.Div(f"Вероятность победы {team2}: {prob_team2:.5f}", style={'fontSize': '20px', 'margin': '10px'})
        ]
    return [prob_text]



@app.callback(
    Output("twitch-iframe", "src"),
    Input("channel-dropdown", "value")
)
def update_twitch_channel(selected_channel):
    if not selected_channel:
        return "https://player.twitch.tv/?channel=bysl4m&parent=localhost&parent=127.0.0.1"
    return f"https://player.twitch.tv/?channel={selected_channel}&parent=localhost&parent=127.0.0.1"


if __name__ == '__main__':
    app.run_server(debug=True) # http://127.0.0.1:8050/
